In [5]:
# The DBLP Publication Network

# 1. Problem statement:

# The following datasets, provided on Canvas, contain bipartite graphs where one set of nodes (vertices) are authors
# and the other set are academic papers. Each edge (a, p) connects an author a to a paper p.
# out.dblp_author-short.txt
# out.dblp_author-medium.txt
# out.dblp_author-long.txt
# out.dblp_author-all.txt
# The short dataset contains 1,000 edges, the medium 10,000 edges, the long version 100,000 edges, and the "all"
# version has all the edges.
# Note: The network is encoded with two numbers per line separated by spaces. You will need to open and read the file,
# and then turn the read data into a meaningful format. If you use Python, you should be able to use built-in method
# split to get a list of numbers. The numbers at even indices (starting at 0) are authors, and the numbers at odd
# indices are publications. You will want to be mindful not to do things that take a lot of operations or memory.

# a) Find the minimum, maximum, and average number authors per paper.
# b) Find the minimum, maximum, and average number of papers per author.
# c) Find the (not necessarily unique) author who has written the most papers. Call this author X. An author other
# than X has an X-index of 1 if she has co-authored at least one paper with X. An author has an X-index of 2 if she
# does not have an X-index of 1, but has co-authored a paper with someone who has an X-index of 1. Similarly, you can
# define having an X-index of 3, 4, etc.
# Write a function that produces the set of authors for some given index, n.

# 2. Mathematical description of the solution:

# The solution can be broken down into three parts:
# (1) creating the adjacency list to represent the network. Each edge (a, p) from an author a to a paper p can be seen
# as an edge in the adjacency list. We can establish two adjacency lists, one using author as the key and set of papers
# for this author as the value, one using paper as key and set of authors for this paper as the value.
# (2) Once the graphs are established, we can calculate the minimum, the maximum, and the average number of papers per
# author and number of authors per paper.
# (3) Using the graph we created, we can obtain a list of X authors by comparing whether the length of the set of papers
# equals to the maximum paper per author. We will use a BFS algorithm to process the graph. The initial array containing
# all the X authors will be the initial queue in the iterative BFS algorithm. Currently, we are at level 0, because X-index
# 1 indicates the co-author of X author, so X author can be set to level 0. We need to process the graph level by level,
# starting from the X author, so this makes sure that we do not go back to the previous level. This logic satisfies the
# problem statements that author with X-index of 2 does not have an X-index of 1, but only co-author with someone who
# has X-index of 1. In order to process the graph level by level, we need to know how many elements are at the current
# level. Thus, we use a variable cur_level_size to save for the number of elements in the queue before we pop anything
# from the queue. For each value that is already in the queue, we label them as visited, so we do not go back to the author
# that we already processed and create an infinite loop. To get the co-author of our current author, we first loop through
# all the papers published by this author, and for each paper, look up all its authors. If these authors are not already
# visited, we can add them to the queue. If the current level has finished processing, increment the level variable by 1
# and go to process the next level. Once the level is equal to the desired X-index of n, we add all the elements in the queue
# into an array and return it.

# 3. Observations and insights drawn from your approach:

# The BFS algorithm is implemented iteratively using a queue data structure. A queue is a FIFO data structure, which means
# that anything added first to the queue will be popped out of the queue first. We are able to process the graph level
# by level by leveraging the characteristics of queue. The time complexity of the algorithm to calculate maximum, minimum,
# and average values are O(V+E). The BFS algorihtm time complexity is O(V+E) as well, so the time complexity of the whole 
# algorithm is O(V+E), where V represents the number of vertex (author and paper), and E represents the number of edges 
# (from author to paper and from paper to author).

# 4. Executable, commented, and clear code:

# See below for the implementation of the algorithm in Python. I have tested the program using the short, medium, and long 
# version, but the program runs really slow on the all version and does not give an input probably limited by my laptop.

import math
from collections import deque
from pathlib import Path

# Read from the publication data file and parse the information to two adjacency list. The first adjacency list
# expressed using two dictionary in Python.
# author_to_paper_adj_list: key is author, value is a set of paper published by this author.
# paper_to_author_adj_list: key is paper, value is a set of authors for this paper.
file_path = Path('out.dblp-author-all.txt')
author_to_paper_adj_list = {}
paper_to_author_adj_list = {}
with open(file_path, 'r') as publication_info_file:
    for line in publication_info_file:
        line_arr = line.split()
        author, paper = line_arr
        author = int(author)
        paper = int(paper)
        if author in author_to_paper_adj_list:
            author_to_paper_adj_list[author].add(paper)
        else:
            author_to_paper_adj_list[author] = {paper}

        if paper in paper_to_author_adj_list:
            paper_to_author_adj_list[paper].add(author)
        else:
            paper_to_author_adj_list[paper] = {author}

# After the two adjacency lists have been created. Go through each one of them, and get the maximum and minimum value
# from each adjacency list. While looping through the dictionary, also keep track of the sum of authors and papers in
# order to calculate the average value after the for loop. The average authors per paper is calculated as the sum of
# number of authors for all the papers divided by the number of papers. The average papers per author is calculated in
# the same manner.
min_papers_per_author = math.inf
max_papers_per_author = -1
sum_of_papers_per_author = 0
for author in author_to_paper_adj_list:
    cur_papers_for_author = len(author_to_paper_adj_list.get(author))
    sum_of_papers_per_author += cur_papers_for_author
    min_papers_per_author = min(cur_papers_for_author, min_papers_per_author)
    max_papers_per_author = max(cur_papers_for_author, max_papers_per_author)


average_papers_per_author = sum_of_papers_per_author / len(author_to_paper_adj_list)

min_authors_per_paper = math.inf
max_authors_per_paper = -1
sum_of_authors_per_paper = 0
for paper in paper_to_author_adj_list:
    cur_authors_for_paper = len(paper_to_author_adj_list.get(paper))
    sum_of_authors_per_paper += cur_authors_for_paper
    min_authors_per_paper = min(cur_authors_for_paper, min_authors_per_paper)
    max_authors_per_paper = max(cur_authors_for_paper, max_authors_per_paper)

average_authors_per_paper = sum_of_authors_per_paper / len(paper_to_author_adj_list)

# Print out the result as two formatted strings.
print("For the file: {}".format("/Users/haotianshen/Desktop/CS5002/CS5002 Final Project/out.dblp-author-all.txt"))
print("The minimum number of authors per paper is {}; The maximum number of authors per paper is {}; The average number of authors per paper is {}.".format(
    min_authors_per_paper, max_authors_per_paper, average_authors_per_paper))
print("The minimum number of papers per author is {}; The maximum number of papers per author is {}; The average number of papers per author is {}.".format(
    min_papers_per_author, max_papers_per_author, average_papers_per_author))


# Function find_all_authors_with_X_index_of_n use the BFS algorithm to find the author with X-index of n. It will take
# in three parameters, X-index of n, author_to_paper adjacency list, paper_to_author adjacency list. A visited array
# will be maintained to keep track of the authors that are already visited because an author with a higher X-index does
# not have a smaller X-index. Process the adjacency list level by level, starting from the level containing author X.
# Once we reach the level of X-index of n, we add the all the authors at this level into an array and return the result.
def find_all_authors_with_X_index_of_n(n, X_authors, author_to_paper_adj_list, paper_to_author_adj_list):
    X_authors = deque(X_authors)
    visited = set()
    authors_with_X_index_of_n = []
    level = 0
    
    for init_author in X_authors:
        if init_author not in visited:
            visited.add(init_author)

    while X_authors:
        cur_level_size = len(X_authors)
        while cur_level_size > 0:
            author = X_authors.popleft()
            visited.add(author)

            if level == n and author not in authors_with_X_index_of_n:
                authors_with_X_index_of_n.append(author)

            for paper in author_to_paper_adj_list[author]:
                co_authors = paper_to_author_adj_list[paper]
                for co_author in co_authors:
                    if co_author not in visited:
                        X_authors.append(co_author)
                        visited.add(co_author)
            cur_level_size = cur_level_size - 1

        level = level + 1

    return authors_with_X_index_of_n

# Function pint_authors_with_X_index_of_n print out the authors with X-index of n to the console. If there is no authors
# with X-index of n, then the function print out a message stating that. Otherwise, the function print out the author
# ID whose X-index is n.
def print_authors_with_X_index_of_n(n, authors_with_X_index_of_n):
    print("The set of authors for X-index of {}".format(n) +
          " are the following author IDs:")
    if len(authors_with_X_index_of_n) == 0:
        print("There is no author with X-index of {}. Please try a smaller X-index value!".format(n))
        return
    for author in authors_with_X_index_of_n:
        print("author ID: {}".format(author))

# Get all the authors with the maximum papers published. These are authors X, which can be either one or more than one.
def get_X_authors(author_to_paper_adj_list, max_papers_per_author):
    X_authors = []
    for author in author_to_paper_adj_list:
        if len(author_to_paper_adj_list.get(author)) == max_papers_per_author:
            X_authors.append(author)
    return X_authors


# Test the code with X-index of 13. 
n_index_1 = 13
X_authors = get_X_authors(author_to_paper_adj_list, max_papers_per_author)
authors_with_X_index_of_n = find_all_authors_with_X_index_of_n(
    n_index_1, X_authors, author_to_paper_adj_list, paper_to_author_adj_list)
authors_with_X_index_of_n.sort()
print_authors_with_X_index_of_n(n_index_1, authors_with_X_index_of_n)
print()

# Commented out for test with X-index of 1 and 2 for the short version.

# n_index_2 = 2
# X_authors = get_X_authors(author_to_paper_adj_list, max_papers_per_author)
# authors_with_X_index_of_n = find_all_authors_with_X_index_of_n(
#     n_index_2, X_authors, author_to_paper_adj_list, paper_to_author_adj_list)
# authors_with_X_index_of_n.sort()
# print_authors_with_X_index_of_n(n_index_2, authors_with_X_index_of_n)
# print()

# n_index_3 = 3
# X_authors = get_X_authors(author_to_paper_adj_list, max_papers_per_author)
# authors_with_X_index_of_n = find_all_authors_with_X_index_of_n(
#     n_index_3, X_authors, author_to_paper_adj_list, paper_to_author_adj_list)
# authors_with_X_index_of_n.sort()
# print_authors_with_X_index_of_n(n_index_3, authors_with_X_index_of_n)
# print()


For the file: /Users/haotianshen/Desktop/CS5002/CS5002 Final Project/out.dblp-author-all.txt
The minimum number of authors per paper is 1; The maximum number of authors per paper is 119; The average number of authors per paper is 2.1621729185155556.
The minimum number of papers per author is 1; The maximum number of papers per author is 951; The average number of papers per author is 6.066024085907479.
The set of authors for X-index of 13 are the following author IDs:
author ID: 1021195
author ID: 1199180
author ID: 1199787

